### Acoustic fingerprinting

The solution will consists of three separate components:
1. Create fingerprints of all songs in the database: Code that creates an unique audio fingerprint of each song in the database
2. Take FFT of a short audio snippet : Code that makes Fourier transforms of a short snippet
3. Matching algorithm: Algorithm that has two functions:
<br>
    3.1. Matches the FFT of the short audio snippet to each of the fingerprints in the database
<br>
    3.2. Returns a list of songs from the database whose fingerprints are closely matching to the short audio snippet

In [98]:
import matplotlib.pyplot as plt
from scipy.fftpack import fft, dct
from scipy.io import wavfile
from scipy.signal import blackman, hanning, hamming
import numpy as np
from scipy import signal
import os

%matplotlib inline

# Number of points to use in the FFT
n_fft = 512

# Frame size and overlap between frames for the FFT
frame_size = 0.025
frame_overlap = 0.015

# Location of the songs
songs_location = "C:/Users/bre49823/Google Drive/MusicEngine/wav/" # "/Users/valentin/Documents/MusicEngine/wav/test/"
songs_list = os.listdir(songs_location)

# Location of sample
sample_file = "C:/Users/bre49823/Google Drive/MusicEngine/wav/dt_16bars_102rap.wav" # "/Users/valentin/Documents/MusicEngine/wav/test/"


In [129]:
########################################
# Create acoustic fingerprints for all songs in the database
########################################

# Create empty lists for song names with FFT frame and the frequency bands
song_fft_window = []
band_0_25 = []
band_26_50 = []
band_51_75 = []
band_76_100 = []
band_101_125 = []
band_126_150 = []
band_151_175 = []
band_176_200 = []
band_201_225 = []
band_226_250 = []


### Loop through all songs in the database for which I want to create a fingerprint
for s in range(0, len(songs_list)):
    
    #### Read in the raw audio data and get the sample rate (in samples/sec)
    sample_rate, soundtrack_data = wavfile.read(songs_location + songs_list[s])

    # If the audio is stereo (len(soundtrack_data.shape) == 2), take only one of the channels, otherwise if mono use as is
    if len(soundtrack_data.shape) == 2:
        audio_signal = soundtrack_data.T[0]
        audio_signal = audio_signal[0:int(2 * sample_rate)]       # keep only the first n seconds
    else:
        audio_signal = soundtrack_data
        audio_signal = audio_signal[0:int(2 * sample_rate)]

    time = np.arange(0, float(audio_signal.shape[0]), 1) / sample_rate


    #### Apply pre-emphasis filter to the raw audio data
    emphasis_coeff = 0.95
    audio_signal = np.append(audio_signal[0], audio_signal[1:] - emphasis_coeff * audio_signal[:-1])


    #### Split the audio data into frames. Fourier Transform needs to be applied over short chunks of the raw audio data
    # Calculate the length of each frame and the step for moving forward the FFT
    frame_stride = round(frame_size - frame_overlap, 3)
    frame_length, frame_step = int(round(frame_size * sample_rate)), int(round(frame_stride * sample_rate))

    # Calculate the total number of frames
    signal_length = len(audio_signal)
    number_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))

    # Pad the raw signal data with zeros to make sure that all frames have equal number of samples
    pad_audio_length = number_frames * frame_step + frame_length          # This number should be very close to the audio signal length. The difference is caused by the rounding in the calculation of number_frames
    zeros_vector = np.zeros((pad_audio_length + signal_length))
    pad_signal = np.append(audio_signal, zeros_vector)


    #### frames_idx is an index which is used to split the pad_signal array
    frames_idx = np.tile(np.arange(0, frame_length), (number_frames, 1)) + np.tile(np.arange(0, number_frames * frame_step, frame_step), (frame_length, 1)).T 
    signal_frames = pad_signal[frames_idx.astype(np.int32, copy = False)]


    #### Create a window function for the Fourier transform. The windows are used for smoothing values of the raw signal in each time frame
    signal_frames *= np.hamming(frame_length)


    #### Calculate FFT (FFT is the implementation of the Discrete Fourier Transformation)
    # The FFT is symmetrical, and by using "np.fft.rfft" we only take the first half automatically. Otherwise, if we use "np.fft.fft" we'll need to take the first half only
    signal_fft_transform = np.fft.rfft(signal_frames, n = n_fft)
    signal_fft_transform_abs = np.absolute(signal_fft_transform)

    # Calculate the power for each frame
    signal_power = ((signal_fft_transform_abs ** 2) / n_fft)

    
    #### Define the length of each frequency bin by deciding how many bins I need and then chunk the output from FFT by each bin
    # When NFFT = 512, the result has 257 datapoints from which I subtract 7 to round the bins
    npoints = signal_fft_transform_abs[0].shape[0] - 7
    frequency_bins = 10
    points_per_bin = npoints / frequency_bins

    #### Create frequency bins from the indices of all frequencies in the range 0 - 250 (for NFFT = 512) in step of 25 
    frames_idx = np.tile(np.arange(0, points_per_bin, 1), (frequency_bins, 1)) + np.tile(np.arange(0, npoints, points_per_bin), (points_per_bin, 1)).T

    # Loop through all the frames/windows to which Fourier transform was applied
    for i in range(0, signal_fft_transform_abs.shape[0]):

        # Limit the output from the Fourier transform only to the first 250 points (for NFFT = 512)
        fft_results = signal_fft_transform_abs[i, 0:npoints]

        # Split the results from the Fourier transform into the frequency bins created above
        fft_results_tiled = fft_results[frames_idx]

        # Calculate the maximum power in each bin. This returns a list with the maximum power for each frequency bin in the window frames of the audio signal
        max_power = [max(fft_results_tiled[x]) for x in range(0, frames_idx.shape[0])]

        # Append the maximum power from each frequency band to the appropriate frequency band lists
        band_0_25.append(max_power[0])
        band_26_50.append(max_power[1])
        band_51_75.append(max_power[2])
        band_76_100.append(max_power[3])
        band_101_125.append(max_power[4])
        band_126_150.append(max_power[5])
        band_151_175.append(max_power[6])
        band_176_200.append(max_power[7])
        band_201_225.append(max_power[8])
        band_226_250.append(max_power[9])

        # Create an index which is a combination of song name and Fourier transform frame. This index tracks songs and frame sequence
        # The number of records in this list should equal the number of records in the lists with frquency bands
        fft_window = i            # A sequential number of the Fourier transform windows for each song
        song_fft_window.append(songs_list[s].split(".wav")[0] + "_" + str(fft_window))


In [43]:
########################################
# Create an acoustic fingerprint for the short song sample 
########################################

#### Read in the raw audio for the sample
sample_rate, soundtrack_data = wavfile.read(sample_file)
audio_signal = soundtrack_data.T[0]                     # this is a two channel soundtrack, get only one of the tracks
audio_signal = audio_signal[0:int(2 * sample_rate)]       # keep only the first n seconds

# Create empty lists for song names with FFT frame and the frequency bands
song_fft_window = []
band_0_25 = []
band_26_50 = []
band_51_75 = []
band_76_100 = []
band_101_125 = []
band_126_150 = []
band_151_175 = []
band_176_200 = []
band_201_225 = []
band_226_250 = []


### Loop through all songs in the database for which I want to create a fingerprint
for s in range(0, len(songs_list)):
    
    #### Read in the raw audio data and get the sample rate (in samples/sec)
    sample_rate, soundtrack_data = wavfile.read(songs_location + songs_list[s])

    # If the audio is stereo (len(soundtrack_data.shape) == 2), take only one of the channels, otherwise if mono use as is
    if len(soundtrack_data.shape) == 2:
        audio_signal = soundtrack_data.T[0]
        audio_signal = audio_signal[0:int(2 * sample_rate)]       # keep only the first n seconds
    else:
        audio_signal = soundtrack_data
        audio_signal = audio_signal[0:int(2 * sample_rate)]

    time = np.arange(0, float(audio_signal.shape[0]), 1) / sample_rate


    #### Apply pre-emphasis filter to the raw audio data
    emphasis_coeff = 0.95
    audio_signal = np.append(audio_signal[0], audio_signal[1:] - emphasis_coeff * audio_signal[:-1])


    #### Split the audio data into frames. Fourier Transform needs to be applied over short chunks of the raw audio data
    # Calculate the length of each frame and the step for moving forward the FFT
    frame_stride = round(frame_size - frame_overlap, 3)
    frame_length, frame_step = int(round(frame_size * sample_rate)), int(round(frame_stride * sample_rate))

    # Calculate the total number of frames
    signal_length = len(audio_signal)
    number_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))

    # Pad the raw signal data with zeros to make sure that all frames have equal number of samples
    pad_audio_length = number_frames * frame_step + frame_length          # This number should be very close to the audio signal length. The difference is caused by the rounding in the calculation of number_frames
    zeros_vector = np.zeros((pad_audio_length + signal_length))
    pad_signal = np.append(audio_signal, zeros_vector)


    #### frames_idx is an index which is used to split the pad_signal array
    frames_idx = np.tile(np.arange(0, frame_length), (number_frames, 1)) + np.tile(np.arange(0, number_frames * frame_step, frame_step), (frame_length, 1)).T 
    signal_frames = pad_signal[frames_idx.astype(np.int32, copy = False)]


    #### Create a window function for the Fourier transform. The windows are used for smoothing values of the raw signal in each time frame
    signal_frames *= np.hamming(frame_length)


    #### Calculate FFT (FFT is the implementation of the Discrete Fourier Transformation)
    # The FFT is symmetrical, and by using "np.fft.rfft" we only take the first half automatically. Otherwise, if we use "np.fft.fft" we'll need to take the first half only
    signal_fft_transform = np.fft.rfft(signal_frames, n = n_fft)
    signal_fft_transform_abs = np.absolute(signal_fft_transform)

    # Calculate the power for each frame
    signal_power = ((signal_fft_transform_abs ** 2) / n_fft)

    
    #### Define the length of each frequency bin by deciding how many bins I need and then chunk the output from FFT by each bin
    # When NFFT = 512, the result has 257 datapoints from which I subtract 7 to round the bins
    npoints = signal_fft_transform_abs[0].shape[0] - 7
    frequency_bins = 10
    points_per_bin = npoints / frequency_bins

    #### Create frequency bins from the indices of all frequencies in the range 0 - 250 (for NFFT = 512) in step of 25 
    frames_idx = np.tile(np.arange(0, points_per_bin, 1), (frequency_bins, 1)) + np.tile(np.arange(0, npoints, points_per_bin), (points_per_bin, 1)).T

    # Loop through all the frames/windows to which Fourier transform was applied
    for i in range(0, signal_fft_transform_abs.shape[0]):

        # Limit the output from the Fourier transform only to the first 250 points (for NFFT = 512)
        fft_results = signal_fft_transform_abs[i, 0:npoints]

        # Split the results from the Fourier transform into the frequency bins created above
        fft_results_tiled = fft_results[frames_idx]

        # Calculate the maximum power in each bin. This returns a list with the maximum power for each frequency bin in the window frames of the audio signal
        max_power = [max(fft_results_tiled[x]) for x in range(0, frames_idx.shape[0])]

        # Append the maximum power from each frequency band to the appropriate frequency band lists
        band_0_25.append(max_power[0])
        band_26_50.append(max_power[1])
        band_51_75.append(max_power[2])
        band_76_100.append(max_power[3])
        band_101_125.append(max_power[4])
        band_126_150.append(max_power[5])
        band_151_175.append(max_power[6])
        band_176_200.append(max_power[7])
        band_201_225.append(max_power[8])
        band_226_250.append(max_power[9])

        # Create an index which is a combination of song name and Fourier transform frame. This index tracks songs and frame sequence
        # The number of records in this list should equal the number of records in the lists with frquency bands
        fft_window = i            # A sequential number of the Fourier transform windows for each song
        song_fft_window.append(songs_list[s].split(".wav")[0] + "_" + str(fft_window))

In [60]:
########################################
# Match the fingerprint of the sample to the database and return the results
# Determine which match is the actual
########################################

# These are test samples
sample_0_25 = band_0_25[0]
sample_26_50 = band_26_50[0]
sample_51_75 = band_51_75[0]
sample_76_100 = band_76_100[0]
sample_101_125 = band_101_125[0]
sample_126_150 = band_126_150[0]
sample_151_175 = band_151_175[0]
sample_176_200 = band_176_200[0]
sample_201_225 = band_201_225[0]
sample_226_250 = band_226_250[0]

from itertools import compress

t = sample_26_50 == band_26_50
match_idx = list(compress(xrange(len(t)), t))

matched_songs = [song_fft_window[i] for i in list(compress(xrange(len(t)), t))]

In [61]:
matched_songs

['bach_partita_e_major_0']

In [137]:
x = 1
for (i = 0; i < 3; i++) {
  x += 5 * i
}
console.log(x)

SyntaxError: invalid syntax (<ipython-input-137-6bac6506a7ca>, line 2)